# Setup environment

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install scikit-bio

In [ ]:
# Clone `CreateDebateScraper` library from github
!git clone https://github.com/utkarsh512/CreateDebateScraper.git
%cd CreateDebateScraper/src/nested/

In [ ]:
from   copy                     import deepcopy
from   itertools                import accumulate
import json
from   matplotlib               import pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import pickle
import re
from   scipy                    import stats
import textwrap
from   thread                   import Comment, Thread
from   tqdm                     import tqdm
nltk.download('punkt') # For tokenizers
import matplotlib
from   nltk.tokenize            import TweetTokenizer
from   pprint                   import pprint
import skbio
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams["figure.figsize"] = (12, 5)

# Helper functions

In [ ]:
tknz = TweetTokenizer()

def clean_text(text):
    """
    Preprocessing text
    """
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub("-", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\u2018", "X", text) 
    text = re.sub("\u2019", "X", text) 
    text = re.sub("\'", "X", text) 
    wordTokens_ = tknz.tokenize(text)
    wordTokens = list()
    for x in wordTokens_:
        x = ''.join([v for v in x if v.isalnum() or v == ' '])
        if len(x) > 0 and x != 'X':
            x = x.replace('X', '\'')
            wordTokens.append(x)
    return wordTokens

In [ ]:
SLUR_WORDS = {
  "jews": [
    "jews",
    "oven dodger",
    "nazi",
    "dirty jew",
    "holocaust",
    "kikesucker",
    "hook nose",
    "kike"
  ],
  "homosexual": [
    "faggots usually",
    "fucking queer",
    "the biggest faggot",
    "dyke",
    "you fucking faggot",
    "hate faggots",
    "queer",
    "homosexual",
    "the faggots",
    "faggot",
    "faggots usually have",
    "gay",
    "faggots",
    "dykey",
    "ugly dyke",
    "faggots like you",
    "you a fag",
    "lesbian",
    "homo",
    "is a faggot",
    "like a faggot",
    "dykes",
    "faggots like",
    "faggot if you ever"
  ],
  "women": [
    "ugly dyke",
    "woman terrorist",
    "nigress",
    "bitch",
    "slut",
    "women",
    "sheeboon",
    "negress",
    "mud shark",
    "women threat",
    "you a lame bitch",
    "your a cunt",
    "white bitch",
    "niggeress",
    "hoe",
    "dykes",
    "niggress",
    "sheboon",
    "feminazi"
  ],
  "blacks": [
    "pavement ape",
    "the niggers",
    "negress",
    "porch monkey",
    "that nigger",
    "this nigger",
    "sheboon",
    "all niggers",
    "eurafrica",
    "shut up nigger",
    "picaninny",
    "african attack",
    "spearchucker",
    "how many niggers",
    "nigger",
    "africa",
    "niggers are in my",
    "dindu nuffin",
    "stupid nigger",
    "moolie",
    "niggers",
    "bluegum",
    "nigger ass",
    "you niggers",
    "fucking nigger",
    "nigger music",
    "niggress",
    "you a nigger",
    "many niggers are",
    "nigress",
    "blacks",
    "teenaper",
    "sheeboon",
    "dumb nigger",
    "niggeress",
    "pickaninny",
    "nigga"
  ],
  "muslim": [
    "muslim immigrant",
    "islam",
    "mudslime",
    "mooslem",
    "muslim refugee",
    "musslime",
    "shitlam",
    "muslim invasion",
    "moslime",
    "mooslamic",
    "muzzie",
    "allah akbar",
    "mooslime",
    "musloid",
    "mudslimes",
    "muslim",
    "muslimes",
    "moslum",
    "mussie",
    "muzrat",
    "muslim countries",
    "muzzy",
    "moslim",
    "jihadi",
    "muslim country",
    "moslem",
    "muzzrat",
    "mooslim"
  ],
  "arabs": [
    "towel head",
    "goatfucker",
    "arabs",
    "goathumper",
    "raghead",
    "rag head",
    "goathumping",
    "towelhead",
    "camel jockey",
    "sandnigger",
    "camel fucker",
    "sand nigger"
  ],
  "generic": [
    "to rape",
    "raped and",
    "shithole country",
    "get raped",
    "raped",
    "is a fucking",
    "shit skin",
    "raped by",
    "hate you",
    "fake empowerment",
    "abusive women",
    "fuck you too",
    "violence",
    "wit a lame nigga",
    "they all look",
    "alllivesmatter",
    "shithole countries",
    "fucking hate",
    "trailer trash",
    "kill all",
    "terrorist threat",
    "harassment",
    "kill yourself",
    "shitskin",
    "okay to be white",
    "fucking hate you"
  ],
  "white": [
    "full of white",
    "white trash",
    "white devil",
    "white",
    "are all white",
    "white boy",
    "white ass",
    "white bitch",
    "hillbilly",
    "whigger",
    "white christian",
    "white person",
    "all white",
    "white nigger",
    "redneck",
    "white honky",
    "wigger",
    "them white"
  ],
  "economy": [
    "ghetto"
  ],
  "immigrant": [
    "illegal immigrants",
    "immigrant not welcome",
    "immigrant terror",
    "mexcrement",
    "go back to where you come from",
    "muslim refugee",
    "illegal aliens",
    "refugee",
    "protect from immigrants",
    "negro",
    "refugees",
    "immigrant",
    "refugee invasion",
    "go back to where they come from",
    "refugees impact",
    "bring ebola",
    "immigrants",
    "illegal alien",
    "immigrant invasion",
    "bring disease"
  ],
  "mental": [
    "retard",
    "mongoloid",
    "retarded"
  ],
  "asians": [
    "asians",
    "ching chong",
    "chinaman"
  ]
}

# Loading CreateDebate dataset

In [ ]:
comments = dict()

# Topical forums on CreateDebate. We have scraped comments for all of the
# following forurm.
categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

# However, we will be analyzing comments from selected forum only!
# These forum have at least 10k comments each.
categories_selected = ['politics2', 'religion', 'world', 
                       'science', 'law', 'technology']

for x in categories_selected:
    comments[x] = list()

In [ ]:
# Loading comments from select forums

for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')

    # Get all the `Thread` objects pickled while scraping.
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()

    # While classifying CreateDebate comments, we used comments as per author mode.
    # Hence, using the same mode to attach classification score with the comments.
    # 
    # score < 0.5 -> ad hominem comment
    #       > 0.5 -> non ad hominem comment
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append(v)
            except:
                authors[v.author] = list()
                authors[v.author].append(v)

    ctr = 0
    # Load the classification score of the comments.
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    # Attach classification score with the comments.
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            foo['score'] = cws[ctr][0]
            foo['validation'] = cws[ctr][1][0]
            comments[cat].append(foo)
            ctr += 1

# Analyzing dataset for slur words across ad hominem score

In [ ]:
ah_score = dict()
slur_count = dict()
slur_count_v2 = dict()

# addressing order: 
#   ah_score: 
#       key: category -> author
#       value: list of score of each comment written
#
#   slur_count:
#       key: category -> slur_group -> author 
#       value: list of slur count of each comment written
#   
#   slur_count_v2:
#       key: category -> slur_group_1 -> slur_group_2 -> author
#       value: list of slur count of each comment written

In [ ]:
# # Computation is already cached.

# for cat in categories_selected:
#     ah_score[cat] = dict()
#     slur_count[cat] = dict()
#     slur_count_v2[cat] = dict()

#     for slur_group in SLUR_WORDS.keys():
#         slur_count[cat][slur_group] = dict()
#         slur_count_v2[cat][slur_group] = dict()
#         for slur_group_ in SLUR_WORDS.keys():
#             slur_count_v2[cat][slur_group][slur_group_] = dict()

#     for comment in tqdm(comments[cat]): 
#         author = comment['author']

#         if author not in ah_score[cat]:
#             ah_score[cat][author] = list()
#             for slur_group in SLUR_WORDS.keys():
#                 slur_count[cat][slur_group][author] = list()
#                 for slur_group_ in SLUR_WORDS.keys():
#                     slur_count_v2[cat][slur_group][slur_group_][author] = list()

#         ah_score[cat][author].append(1 - comment['score'])
#         cleaned_comment_text = clean_text(comment['body'])

#         for slur_group, slur_words in SLUR_WORDS.items():
#             current_count = sum([cleaned_comment_text.count(slur_word) for slur_word in slur_words])
#             slur_count[cat][slur_group][author].append(current_count)

#         for slur_group_1 in SLUR_WORDS.keys():
#             for slur_group_2 in SLUR_WORDS.keys():
#                 if slur_group_1 == slur_group_2:
#                     continue
#                 current_count = sum(
#                     [
#                         cleaned_comment_text.count(slur_word_1 + ' ' + slur_word_2)
#                         for slur_word_1 in SLUR_WORDS[slur_group_1]
#                         for slur_word_2 in SLUR_WORDS[slur_group_2]
#                     ]
#                 )
#                 slur_count_v2[cat][slur_group_1][slur_group_2][author].append(current_count)           

In [ ]:
# # Caching computation

# with open('/content/gdrive/MyDrive/Temp/47-ah-score.pkl', 'wb') as fp:
#     pickle.dump(ah_score, fp)

# with open('/content/gdrive/MyDrive/Temp/47-slur-count.pkl', 'wb') as fp:
#     pickle.dump(slur_count, fp)

# with open('/content/gdrive/MyDrive/Temp/47-slur-count-v2.pkl', 'wb') as fp:
#     pickle.dump(slur_count_v2, fp)

In [ ]:
# Loading computation from cache

with open('/content/gdrive/MyDrive/Temp/47-ah-score.pkl', 'rb') as fp:
    ah_score = pickle.load(fp)

with open('/content/gdrive/MyDrive/Temp/47-slur-count.pkl', 'rb') as fp:
    slur_count = pickle.load(fp)

with open('/content/gdrive/MyDrive/Temp/47-slur-count-v2.pkl', 'rb') as fp:
    slur_count_v2 = pickle.load(fp)

In [ ]:
# Testing where v2 are computed correctly!

pair_count = 0

for category in categories_selected:
    for slur_group_1 in SLUR_WORDS.keys():
        for slur_group_2 in SLUR_WORDS.keys():
            for lst in slur_count_v2[category][slur_group_1][slur_group_2].values():
                pair_count += sum(lst)

print(pair_count)

In [ ]:
# Median ah score per category per author
#   key: category -> author
#   value: median ah score
ah_score_median = dict()

for category, author_data in ah_score.items():
    ah_score_median[category] = dict()
    for author, ah_scores in author_data.items():
        ah_score_median[category][author] = np.median(ah_scores)

In [ ]:
comment_count = dict()
# key: category -> author
# value: number of comments written by author in the given forum

for category in categories_selected:
    comment_count[category] = dict()

    for comment in comments[category]:
        author = comment['author']
        try:
            comment_count[category][author] += 1
        except KeyError:
            comment_count[category][author] = 1

In [ ]:
IGNORE_POLITICS = True
IGNORE_RELIGION = True

EXTREME_AH_UPPER = 1
EXTREME_AH_LOWER = 0.95

MODERATE_AH_UPPER = 0.8
MODERATE_AH_LOWER = 0.7

LOW_AH_UPPER = 0.6
LOW_AH_LOWER = 0.5

extreme_ah_group = dict()
moderate_ah_group = dict()
low_ah_group = dict()
# key: category -> slur_group
# value: list of slur_count

extreme_ah_group_v2 = dict()
moderate_ah_group_v2 = dict()
low_ah_group_v2 = dict()
# key: category -> slur_group_1 -> slur_group_2
# value: list of slur_count

In [ ]:
for category in categories_selected:
    extreme_ah_group[category] = dict()
    moderate_ah_group[category] = dict()
    low_ah_group[category] = dict()

    extreme_ah_group_v2[category] = dict()
    moderate_ah_group_v2[category] = dict()
    low_ah_group_v2[category] = dict()

    for slur_group in SLUR_WORDS.keys():
        extreme_ah_group[category][slur_group] = list()
        moderate_ah_group[category][slur_group] = list()
        low_ah_group[category][slur_group] = list()

        extreme_ah_group_v2[category][slur_group] = dict()
        moderate_ah_group_v2[category][slur_group] = dict()
        low_ah_group_v2[category][slur_group] = dict()

        for slur_group_ in SLUR_WORDS.keys():
            extreme_ah_group_v2[category][slur_group][slur_group_] = list()
            moderate_ah_group_v2[category][slur_group][slur_group_] = list()
            low_ah_group_v2[category][slur_group][slur_group_] = list()

    for author, median_ah_score in ah_score_median[category].items():
        n_politics_cnt = 0
        n_religion_cnt = 0
        try:
            n_politics_cnt = comment_count['politics2'][author]
        except KeyError:
            pass
        try:
            n_religion_cnt = comment_count['religion'][author]
        except KeyError:
            pass

        if IGNORE_POLITICS and n_politics_cnt > 0:
            continue

        if IGNORE_RELIGION and n_religion_cnt > 0:
            continue

        if LOW_AH_LOWER <= median_ah_score and median_ah_score <= LOW_AH_UPPER:
            for slur_group in SLUR_WORDS.keys():
                low_ah_group[category][slur_group].append(np.sum(slur_count[category][slur_group][author]) / len(ah_score[category][author]))
                for slur_group_ in SLUR_WORDS.keys():
                    if slur_group == slur_group_:
                        continue
                    low_ah_group_v2[category][slur_group][slur_group_].append(np.sum(slur_count_v2[category][slur_group][slur_group_][author]) / len(ah_score[category][author]))

        elif MODERATE_AH_LOWER <= median_ah_score and median_ah_score <= MODERATE_AH_UPPER:
            for slur_group in SLUR_WORDS.keys():
                moderate_ah_group[category][slur_group].append(np.sum(slur_count[category][slur_group][author]) / len(ah_score[category][author]))
                for slur_group_ in SLUR_WORDS.keys():
                    if slur_group == slur_group_:
                        continue
                    moderate_ah_group_v2[category][slur_group][slur_group_].append(np.sum(slur_count_v2[category][slur_group][slur_group_][author]) / len(ah_score[category][author]))
                
        elif EXTREME_AH_LOWER <= median_ah_score and median_ah_score <= EXTREME_AH_UPPER:
            for slur_group in SLUR_WORDS.keys():
                extreme_ah_group[category][slur_group].append(np.sum(slur_count[category][slur_group][author]) / len(ah_score[category][author]))
                for slur_group_ in SLUR_WORDS.keys():
                    if slur_group == slur_group_:
                        continue
                    extreme_ah_group_v2[category][slur_group][slur_group_].append(np.sum(slur_count_v2[category][slur_group][slur_group_][author]) / len(ah_score[category][author]))
        

In [ ]:
for category in categories_selected:
    for slur_group_1 in SLUR_WORDS.keys():
        for slur_group_2 in SLUR_WORDS.keys():
            avg_extreme = sum(extreme_ah_group_v2[category][slur_group_1][slur_group_2])
            avg_moderate = sum(moderate_ah_group_v2[category][slur_group_1][slur_group_2])
            avg_low = sum(low_ah_group_v2[category][slur_group_1][slur_group_2])
            print(f'{category:>20} - {slur_group_1:>15} - {slur_group_2:>15} - {[avg_extreme, avg_moderate, avg_low]}')

In [ ]:
# How many users considered for each case

report = ' ' * 20 + ', '
for category in categories_selected:
    report += f'{category:>20}, '
for slur_group in SLUR_WORDS.keys():
    report += '\n' + f'{slur_group:20}, '
    for category in categories_selected:
        s = f'{len(extreme_ah_group[category][slur_group])} / {len(moderate_ah_group[category][slur_group])} / {len(low_ah_group[category][slur_group])}'
        report += f'{s:>20}, '
print(report)

In [ ]:
def plot_category(category, saveas='sample.eps', func=np.average):
    x = list(SLUR_WORDS.keys())
    y1 = []
    y2 = []
    y3 = []

    for slur_group in x:
        y1.append(func(extreme_ah_group[category][slur_group]))
        y2.append(func(moderate_ah_group[category][slur_group]))
        y3.append(func(low_ah_group[category][slur_group]))

    ticks = np.arange(len(x))
    width = 0.20

    fig, ax = plt.subplots()
    fig.set_size_inches(15, 12)
    subplot1 = ax.bar(ticks - width, y1, width, label='Extreme', tick_label=x)
    subplot2 = ax.bar(ticks, y2, width, label='Moderate', tick_label=x)
    subplot3 = ax.bar(ticks + width, y3, width, label='Low', tick_label=x)
    ax.set_ylabel('Slur count')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=30, ha='right')
    ax.legend()
    plt.savefig(saveas)

def plot_slurs(slur_group, saveas='sample.eps', func=np.average):
    x = categories_selected
    y1 = []
    y2 = []
    y3 = []

    for category in x:
        y1.append(func(extreme_ah_group[category][slur_group]))
        y2.append(func(moderate_ah_group[category][slur_group]))
        y3.append(func(low_ah_group[category][slur_group]))

    ticks = np.arange(len(x))
    width = 0.20

    fig, ax = plt.subplots()
    subplot1 = ax.bar(ticks - width, y1, width, label='Extreme', tick_label=x)
    subplot2 = ax.bar(ticks, y2, width, label='Moderate', tick_label=x)
    subplot3 = ax.bar(ticks + width, y3, width, label='Low', tick_label=x)
    ax.set_ylabel('Slur count')
    ax.set_title(f'Slur count distribution for \'{slur_group}\' slurs')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.legend()
    plt.show()
    plt.savefig(saveas, format='eps')

def plot_slurs_v2(slur_group_1, slur_group_2, func=np.average):
    x = categories_selected
    y1 = []
    y2 = []
    y3 = []

    for category in x:
        y1.append(func(extreme_ah_group_v2[category][slur_group_1][slur_group_2]))
        y2.append(func(moderate_ah_group_v2[category][slur_group_1][slur_group_2]))
        y3.append(func(low_ah_group_v2[category][slur_group_1][slur_group_2]))

    ticks = np.arange(len(x))
    width = 0.20

    fig, ax = plt.subplots()
    subplot1 = ax.bar(ticks - width, y1, width, label='Extreme', tick_label=x)
    subplot2 = ax.bar(ticks, y2, width, label='Moderate', tick_label=x)
    subplot3 = ax.bar(ticks + width, y3, width, label='Low', tick_label=x)
    ax.set_ylabel('Slur count')
    ax.set_title(f'Slur count distribution for \'{slur_group}\' slurs')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.legend()
    plt.show()

In [ ]:
plot_category('politics2', '/content/gdrive/MyDrive/Temp/MTPImages/politics_slur_count_2.pdf')

In [ ]:
plot_category('religion', '/content/gdrive/MyDrive/Temp/MTPImages/religion_slur_count_2.pdf')

In [ ]:
plot_category('world', '/content/gdrive/MyDrive/Temp/MTPImages/world_slur_count_2.pdf')

In [ ]:
plot_category('science', '/content/gdrive/MyDrive/Temp/MTPImages/science_slur_count_2.pdf')

In [ ]:
plot_category('law', '/content/gdrive/MyDrive/Temp/MTPImages/law_slur_count_2.pdf')

In [ ]:
plot_category('technology', '/content/gdrive/MyDrive/Temp/MTPImages/technology_slur_count_2.pdf')

In [ ]:
plot_slurs('jews')

In [ ]:
plot_slurs('homosexual')

In [ ]:
plot_slurs('women')

In [ ]:
plot_slurs('blacks')

In [ ]:
plot_slurs('muslim')

In [ ]:
plot_slurs('arabs')

In [ ]:
plot_slurs('generic')

In [ ]:
plot_slurs('white')

In [ ]:
plot_slurs('economy')

In [ ]:
plot_slurs('immigrant')

In [ ]:
plot_slurs('mental')

In [ ]:
plot_slurs('asians')

# Analyzing dataset for slur words across diversity

In [ ]:
user_list = set()

for category in categories_selected:
    for comment in comments[category]:
        user_list.add(comment['author'])

user_list = list(user_list)

In [ ]:
len(user_list)

In [ ]:
ah_activity_matrix = [[0 for j in range(4)] for i in range(len(user_list))]

In [ ]:
for i, user in enumerate(user_list):
    for j, category in enumerate(categories_selected):
        try:
            current_score = ah_score_median[category][user]
            if EXTREME_AH_LOWER <= current_score and current_score <= EXTREME_AH_UPPER:
                ah_activity_matrix[i][1] += 1
            elif MODERATE_AH_LOWER <= current_score and current_score <= MODERATE_AH_UPPER:
                ah_activity_matrix[i][2] += 1
            elif LOW_AH_LOWER <= current_score and current_score <= LOW_AH_UPPER:
                ah_activity_matrix[i][3] += 1
            else:
                ah_activity_matrix[i][0] += 1
        except KeyError:
            ah_activity_matrix[i][0] += 1
            pass

In [ ]:
diversity_map = dict()
# key: author
# value: diversity of the user

# Note that we include only those users who are present in the following categories for at least one topical forum:
# * EXTREME_AH
# * MODERATE_AH
# * LOW_AH

for i, user in enumerate(user_list):

    # Check whether to include this user in the study or not
    flag = ah_activity_matrix[i][1] or ah_activity_matrix[i][2] or ah_activity_matrix[i][3]
    if not flag:
        continue
    
    # It's okay to include this user in the study
    div = skbio.diversity.alpha.shannon(ah_activity_matrix[i]) / 2 # divided by 2 to normalize `div` in [0, 1] range
    diversity_map[user] = div

In [ ]:
len(diversity_map)

In [ ]:
plt.hist(diversity_map.values(), bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
plt.xlabel('Normalized Shannon diversity index')
plt.ylabel('#Users (in log-scale)')
plt.yscale('log')
plt.title('Normalized Shannon diversity index for CreateDebate users')

In [ ]:
# Thresholds to partition these users as 
# - highly diverse
# - moderately diverse

HIGH_DIVERSITY_LOWER = 0.6

MODERATE_DIVERSITY_LOWER = 0.3
MODERATE_DIVERSITY_UPPER = 0.4

high_diversity_users = set()
moderate_diversity_users = set()

In [ ]:
for user, d in diversity_map.items():
    if d >= HIGH_DIVERSITY_LOWER:
        high_diversity_users.add(user)
    elif d >= MODERATE_DIVERSITY_LOWER and d <= MODERATE_DIVERSITY_UPPER:
        moderate_diversity_users.add(user)

In [ ]:
print("Number of highly diverse users: ", len(high_diversity_users))
print("Number of moderately diverse users: ", len(moderate_diversity_users))

In [ ]:
high_diversity_slur_count = dict()
moderate_diversity_slur_count = dict() 
# key: category -> slur_group
# value: list of slur count for every comment written by users belonging to that group

In [ ]:
for category in categories_selected:
    high_diversity_slur_count[category] = dict()
    moderate_diversity_slur_count[category] = dict()

    for slur_group in SLUR_WORDS.keys():
        high_diversity_slur_count[category][slur_group] = list()
        moderate_diversity_slur_count[category][slur_group] = list()

        for user in high_diversity_users:
            try:
                high_diversity_slur_count[category][slur_group].extend(slur_count[category][slur_group][user])
            except KeyError:
                pass
        
        for user in moderate_diversity_users:
            try:
                moderate_diversity_slur_count[category][slur_group].extend(slur_count[category][slur_group][user])
            except KeyError:
                pass

In [ ]:
def plot_category(category, func=np.average):
    x = list(SLUR_WORDS.keys())
    y1 = []
    y2 = []

    for slur_group in x:
        y1.append(func(high_diversity_slur_count[category][slur_group]))
        y2.append(func(moderate_diversity_slur_count[category][slur_group]))

    ticks = np.arange(len(x))
    width = 0.35

    fig, ax = plt.subplots()
    subplot1 = ax.bar(ticks - width / 2, y1, width, label='High', tick_label=x)
    subplot2 = ax.bar(ticks + width / 2, y2, width, label='Moderate', tick_label=x)
    ax.set_ylabel('Slur count')
    ax.set_title(f'Slur count distribution for \'{category}\' forum')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.legend()
    plt.show()

def plot_slurs(slur_group, func=np.average):
    x = categories_selected
    y1 = []
    y2 = []

    for category in x:
        y1.append(func(high_diversity_slur_count[category][slur_group]))
        y2.append(func(moderate_diversity_slur_count[category][slur_group]))

    ticks = np.arange(len(x))
    width = 0.35

    fig, ax = plt.subplots()
    subplot1 = ax.bar(ticks - width / 2, y1, width, label='High', tick_label=x)
    subplot2 = ax.bar(ticks + width / 2, y2, width, label='Moderate', tick_label=x)
    ax.set_ylabel('Slur count')
    ax.set_title(f'Slur count distribution for \'{slur_group}\' slurs')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.legend()
    plt.show()

In [ ]:
plot_category('politics2')

In [ ]:
plot_category('religion')

In [ ]:
plot_category('world')

In [ ]:
plot_category('science')

In [ ]:
plot_category('law')

In [ ]:
plot_category('technology')

In [ ]:
categories_selected

# Temporal variation in diversity of users

In [ ]:
comments['politics2'][0]

In [ ]:
diversity = dict()
# key: author -> year
# value: diversity value

ah_score_list = dict()
# key: author -> year -> category
# value: ah score for comments

In [ ]:
for user in user_list:
    ah_score_list[user] = dict()
    for year in range(2008, 2022):
        syear = str(year)
        ah_score_list[user][syear] = dict()
        for category in categories_selected:
            ah_score_list[user][syear][category] = list()

In [ ]:
for category in categories_selected:
    for comment in tqdm(comments[category]):
        year = comment['time'][:4]
        try:
            int(year)
        except:
            # Time is not available for given comment
            continue
        author = comment['author']
        score = 1 - comment['score']
        ah_score_list[author][year][category].append(score)

In [ ]:
for user in tqdm(user_list):
    if not (user in high_diversity_users or user in moderate_diversity_users):
        continue
    diversity[user] = dict()
    for year in range(2008, 2022):
        syear = str(year)
        freq = [0, 0, 0, 0]
        for category in categories_selected:
            try:
                median_score = np.median(ah_score_list[user][syear][category])
                if median_score >= EXTREME_AH_LOWER and median_score <= EXTREME_AH_UPPER:
                    freq[1] += 1
                elif median_score >= MODERATE_AH_LOWER and median_score <= MODERATE_AH_UPPER:
                    freq[2] += 1
                elif median_score >= LOW_AH_LOWER and median_score <= LOW_AH_UPPER:
                    freq[3] += 1
                else:
                    freq[0] += 1
            except KeyError:
                freq[0] += 1
        diversity[user][syear] = skbio.diversity.alpha.shannon(freq) / 2

In [ ]:
def plot_diversity_user(user):
    x = [str(year) for year in range(2008, 2022)]
    y = [diversity[user][year] for year in x]
    plt.plot(x, y)
    plt.xlabel('Year')
    plt.ylabel('Diversity')
    plt.title(f'Shannon diveristy index for user:{user}')

In [ ]:
list(moderate_diversity_users)[5:10]

In [ ]:
plot_diversity_user('EmoKillMeNow')

In [ ]:
def plot_diversity_users():
    x = [str(year) for year in range(2008, 2022)]
    y1 = [np.average([diversity[user][year] for user in high_diversity_users]) for year in x]
    y2 = [np.average([diversity[user][year] for user in moderate_diversity_users]) for year in x]
    plt.plot(x, y1, label='High')
    plt.plot(x, y2, label='Moderate')
    plt.xlabel('Year')
    plt.ylabel('Diversity')
    plt.legend()
    plt.title(f'Avg. Shannon diveristy index for highly and moderately diverse group')

In [ ]:
plot_diversity_users()

In [ ]:
def plot_moderate_diversity_users():
    x = [str(year) for year in range(2008, 2022)]
    y = [np.average([diversity[user][year] for user in high_diversity_users]) for year in x]
    plt.plot(x, y)
    plt.xlabel('Year')
    plt.ylabel('Diversity')
    plt.title(f'Avg. Shannon diveristy index for overall high diversity users')